## 환경설정

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth()  # 처음 실행 시 브라우저 인증
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1071344378062-kv36e6mgl1vnr89evq958br79d063h8p.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
google_dirve_md_id = os.getenv("GOOGLE_DRIVE_MD_ID")

## 문서 로드

In [3]:
# .md 파일만 검색
file_list = drive.ListFile({
    'q': f"'{google_dirve_md_id}' in parents and title contains '.md' and trashed=false"
}).GetList()
len(file_list)

43

In [4]:
from langchain_core.documents import Document

# 자막 데이터 로드
docs = []
for file in file_list:
    source = file['title']
    content = file.GetContentString()
    doc = Document(
        page_content=content, 
        metadata={"source": source}
    )
    docs.append(doc)

    print(f'{source} : {len(content)}')
    print('---'*10)

[즉문즉설] 제83회 신랑이 바람을 피웠어요..ko.md : 639
------------------------------
(EN/FR/中文 SUB) 제314회 전생과 윤회에 대하여.ko.md : 1197
------------------------------
[즉문즉설]딸이 정토회에 너무 빠져있어서 화가 날 정도예요.ko.md : 1035
------------------------------
버티는 대통령, 어떻게 이해해요?.ko.md : 1851
------------------------------
(ENG /中文 SUB)  법륜스님 즉문즉설 1203회 행복은 마음먹기에 따라 달라진다.ko.md : 1592
------------------------------
부부가 떨어져 사는 것이 옳은지요?.ko.md : 1522
------------------------------
제336회 부익부 빈익빈의 현상.ko.md : 862
------------------------------
남편이 짠돌이.ko.md : 553
------------------------------
결혼을 못했습니다.ko.md : 1374
------------------------------
장인에게 혼난 후 아내까지 미워요.ko.md : 786
------------------------------
(Eng/中文-SUB) 즉문즉설1026회 여자친구 몰래 바람을 피웠는데 돌아가야할지 고민이에요.ko.md : 591
------------------------------
제553회 헤어진 남친을 다시 만나고 있는데, 그만 헤어져야 할 지 만나야 할 지.ko.md : 3031
------------------------------
[즉문즉설]이혼 도장은 찍었지만 마음이 답답합니다.ko.md : 1005
------------------------------
제513회 남을 바꾸려고 하지마라(법문).ko.md : 565
------------------------------
(EN/中

## 문서 인덱싱

In [3]:
from langchain_openai.embeddings import OpenAIEmbeddings

# 임베딩 정의
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    dimensions=1024
)

# 문서 임베딩
# document_embeddings = embeddings.embed_documents(
#     [chunk.page_content for chunk in docs]
# )

In [9]:
from langchain.vectorstores import Chroma

# 벡터 스토어 저장
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    collection_name="content-250618",
    persist_directory="./chroma_db"
)

In [5]:
from langchain.vectorstores import Chroma

# 벡터 스토어 로드
vectorstore = Chroma(
    embedding_function=embeddings,
    collection_name="content-250618",
    persist_directory="./chroma_db"
)

C:\Users\CNXK\AppData\Local\Temp\ipykernel_33036\3448756615.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [8]:
from pprint import pprint

docs = vectorstore.get()
pprint(docs['ids'][0])
pprint(docs['metadatas'][0])
pprint(docs['documents'][0])

'59068932-72c6-4ad7-bd2f-5bf54d039ace'
{'source': '[즉문즉설] 제83회 신랑이 바람을 피웠어요..ko.md'}
('질문자: 남편이 바람을 피워서 너무 힘듭니다. 용서도 안 되고 짜증만 나는데 어떻게 해야 할까요?\n'
 '\n'
 '스님: 세 가지 방법이 있습니다. 첫째, 그냥 헤어지세요. 요즘 세상에 바람피우는 남자랑 살 이유가 없어요. 둘째, 헤어지기 어렵다면 '
 '남편 문제를 내 문제로 돌려놓고, 남편의 장단점을 따져 선택하세요. 셋째, 남편이 왜 바람을 피우는지 연구해 보세요. 심리적인 결함이 '
 '있을 수도 있습니다.\n'
 '\n'
 '질문자: 애 때문에, 시부모님 때문에, 돈 때문에 헤어지기 어렵습니다.\n'
 '\n'
 '스님: 그럼 남편의 장점을 보고 단점을 감수할지 선택해야 합니다. 남편을 미워하지 말고, 버리면 주워갈 사람이 있다는 자신감을 '
 '가지세요.\n'
 '\n'
 '스님: 남편이 정말 나를 싫어해서 바람을 피우는 거라면 헤어지는 게 맞지만, 그게 아니라면 남편의 심리를 연구해 보세요. 어떤 여자를 '
 '만나는지, 왜 만나는지 분석해서 논문이라도 쓰세요.\n'
 '\n'
 '스님: 남편에게 필요한 여러 역할을 내가 다 해줄 수 없다면, 역할을 분담하든지 헤어지든지 해야 합니다. 미워하고 원망하는 건 문제 '
 '해결에 도움이 안 돼요. 특히 아이들에게 안 좋습니다.\n'
 '\n'
 '스님: 용서가 안 되고 짜증이 나면 이미 병입니다. 생각을 바꾸고 남편을 이해하려고 노력해야 합니다. 남편에 대해 연구하고, 사랑이 '
 '결핍된 건 아닌지 살펴보세요.')


In [15]:
# 검색 테스트 
query = "불안해요."

retriever_k = vectorstore.as_retriever(
    search_kwargs={"k": 3},
)
retrieved_docs = retriever_k.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(doc.metadata['source'])
    print('---'*20)
    print(doc.page_content[:100])
    print('==='*20)

쿼리: 불안해요.
검색 결과:
제553회 헤어진 남친을 다시 만나고 있는데, 그만 헤어져야 할 지 만나야 할 지.ko.md
------------------------------------------------------------
질문자: 10년 된 남친이 있었는데, 2년 전 헤어지고 다시 만나요. 집안 환경 때문에 남친 부모님이 안 좋아하셔서 헤어졌는데, 저희 집이 매일 싸우는 환경이라 저도 행복하지 못했
(EN/日本語/FR/中文-SUB) [법륜스님의 즉문즉설 제 1516회] 선택의 기로.ko.md
------------------------------------------------------------
질문자: 스님 뵈니 너무 좋습니다. 인생은 선택의 기로인데, 후회를 최소화하는 방법이 궁금합니다. 예전에 조선소 차장님이 힘든 시기에 못 버틴 걸 엄청 후회하시더라고요. 저도 5년
제336회 부익부 빈익빈의 현상.ko.md
------------------------------------------------------------
질문자: 스님, 안녕하세요. 요즘 일하는 게 너무 힘듭니다. 부익부 빈익빈 현상을 어떻게 이해하고 어떤 마음가짐으로 살아야 할까요?

스님: 괴로움은 크게 두 종류인데, 첫째는 가


In [ ]:
# 검색 테스트 
query = "불안해요."

retriever_mmr = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k': 3,                 # 검색할 문서의 수
        'fetch_k': 8,           # mmr 알고리즘에 전달할 문서의 수 (fetch_k > k)
        'lambda_mult': 0.3,     # 다양성을 고려하는 정도 (1은 최소 다양성, 0은 최대 다양성을 의미. 기본값은 0.5)
        },
)
retrieved_docs = retriever_mmr.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(doc.metadata['source'])
    print('---'*20)
    print(doc.page_content[:100])
    print('==='*20)


쿼리: 불안해요.
검색 결과:
제553회 헤어진 남친을 다시 만나고 있는데, 그만 헤어져야 할 지 만나야 할 지.ko.md
------------------------------------------------------------
질문자: 10년 된 남친이 있었는데, 2년 전 헤어지고 다시 만나요. 집안 환경 때문에 남친 부모님이 안 좋아하셔서 헤어졌는데, 저희 집이 매일 싸우는 환경이라 저도 행복하지 못했
제336회 부익부 빈익빈의 현상.ko.md
------------------------------------------------------------
질문자: 스님, 안녕하세요. 요즘 일하는 게 너무 힘듭니다. 부익부 빈익빈 현상을 어떻게 이해하고 어떤 마음가짐으로 살아야 할까요?

스님: 괴로움은 크게 두 종류인데, 첫째는 가
(EN/FR/中文-SUB)법륜스님 즉문즉설 깨달음과 죽음에 대해 알고 싶어요..ko.md
------------------------------------------------------------
질문자: 깨달음과 죽음에 대해 알고 싶습니다. 죽은 후에는 어떻게 되는지 궁금합니다.
스님: 죽어보면 알게 될 텐데 뭘 미리 물어요? 죽음이라는 건 살아있는 삶을 제대로 아는가에 


In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.1
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever_mmr,
    return_source_documents=True
)

# 질의 테스트
query = "불안해요."
result = qa_chain.invoke(query)
print(result)

{'query': '불안해요.', 'result': '불안한 마음이 드시는군요.\n\n스님께서는 불안한 마음이 들 때 다음과 같이 말씀하셨습니다.\n\n*   **현재에 집중하세요:** "항상 현재에 깨어있어야 합니다. 숨이 들어오고 나가는 것만 알아차리세요. 지나간 일이나 미래를 걱정하지 마세요." 지금 이 순간에 집중하고, 현재 주어진 삶에 감사하며 행복하게 살아가라고 하셨습니다.\n*   **욕심을 내려놓으세요:** "바램이 다 될 수 없기 때문이죠. ... 원하는 게 안 된다고 괴로워할 이유가 없다는 거예요. 이루어지는 게 좋을지, 안 이루어지는 게 좋을지 모르니까요. 그냥 담담하게 받아들이면 인생을 가볍게 살아갈 수 있어요." 모든 것이 뜻대로 되지 않는다고 괴로워할 필요가 없으며, 담담하게 받아들이는 것이 중요하다고 하셨습니다.\n*   **실패를 두려워하지 마세요:** "실패는 좋은 거예요. 실패를 통해 연구하고 발전할 수 있으니까요." 실패는 흉이 아니라 교훈이 되어 약이 될 수 있으니, 두려워하지 말고 교훈 삼아 나아가라고 하셨습니다.\n*   **마음을 다스리세요:** "꾸준히 기도하면서 자기를 돌이켜보고 마음을 잘 다스려야 합니다." 특히 108배 절을 하며 부모님께 감사 기도를 하는 것을 권하셨습니다.\n\n불안한 마음이 들 때, 스님의 말씀처럼 현재에 집중하고, 모든 것을 담담하게 받아들이며, 꾸준히 마음을 다스리는 노력을 해보시는 것이 좋겠습니다.', 'source_documents': [Document(metadata={'source': '제553회 헤어진 남친을 다시 만나고 있는데, 그만 헤어져야 할 지 만나야 할 지.ko.md'}, page_content='질문자: 10년 된 남친이 있었는데, 2년 전 헤어지고 다시 만나요. 집안 환경 때문에 남친 부모님이 안 좋아하셔서 헤어졌는데, 저희 집이 매일 싸우는 환경이라 저도 행복하지 못했어요. 지금도 환경은 그대로고, 남친과 어색한 사이로 만나요. 다시 만나니 좋지만, 한 번 헤어지니

In [14]:
from IPython.display import Markdown, display

display(Markdown(result['result']))

불안한 마음이 드시는군요.

스님께서는 불안한 마음이 들 때 다음과 같이 말씀하셨습니다.

*   **현재에 집중하세요:** "항상 현재에 깨어있어야 합니다. 숨이 들어오고 나가는 것만 알아차리세요. 지나간 일이나 미래를 걱정하지 마세요." 지금 이 순간에 집중하고, 현재 주어진 삶에 감사하며 행복하게 살아가라고 하셨습니다.
*   **욕심을 내려놓으세요:** "바램이 다 될 수 없기 때문이죠. ... 원하는 게 안 된다고 괴로워할 이유가 없다는 거예요. 이루어지는 게 좋을지, 안 이루어지는 게 좋을지 모르니까요. 그냥 담담하게 받아들이면 인생을 가볍게 살아갈 수 있어요." 모든 것이 뜻대로 되지 않는다고 괴로워할 필요가 없으며, 담담하게 받아들이는 것이 중요하다고 하셨습니다.
*   **실패를 두려워하지 마세요:** "실패는 좋은 거예요. 실패를 통해 연구하고 발전할 수 있으니까요." 실패는 흉이 아니라 교훈이 되어 약이 될 수 있으니, 두려워하지 말고 교훈 삼아 나아가라고 하셨습니다.
*   **마음을 다스리세요:** "꾸준히 기도하면서 자기를 돌이켜보고 마음을 잘 다스려야 합니다." 특히 108배 절을 하며 부모님께 감사 기도를 하는 것을 권하셨습니다.

불안한 마음이 들 때, 스님의 말씀처럼 현재에 집중하고, 모든 것을 담담하게 받아들이며, 꾸준히 마음을 다스리는 노력을 해보시는 것이 좋겠습니다.

In [16]:
retrieved_docs

[Document(metadata={'source': '제553회 헤어진 남친을 다시 만나고 있는데, 그만 헤어져야 할 지 만나야 할 지.ko.md'}, page_content='질문자: 10년 된 남친이 있었는데, 2년 전 헤어지고 다시 만나요. 집안 환경 때문에 남친 부모님이 안 좋아하셔서 헤어졌는데, 저희 집이 매일 싸우는 환경이라 저도 행복하지 못했어요. 지금도 환경은 그대로고, 남친과 어색한 사이로 만나요. 다시 만나니 좋지만, 한 번 헤어지니 확실한 답을 받고 싶고, 스님 책 보니 부모님 반대가 심하면 헤어져야 할 것 같기도 해요. 10년 동안 죽고 못 살 정도는 아니었지만, 익숙함과 편안함에 좋아해요. 남친에 대해 어떤 마음을 가져야 할지, 부모님께는 어떻게 해야 할지 궁금합니다.\n\n스님: 남친은 내버려두고 300일 기도를 해봐요. 108배 하면서 "부모님 감사합니다" 이렇게 말해요. 그러면 남친이 떠나가든지, 아니면 관계가 정리될 거예요. 헤어져야 한다, 만나야 한다 생각하지 말고, 다가오면 거절 말고, 멀어지면 잡지 말고 미지근하게 대하세요. 10년 됐으면 원래 미지근한 게 좋은 거예요. 법문 들을 때 항상 자신에게 적용하면 안 돼요.\n\n질문자: 네.\n\n스님: 예를 들어 남자가 재혼 가정이라 부모님이 반대한다고 하면, "부모 핑계 대지 마라, 네가 확실하지 않아서 그렇다"라고 말할 수 있어요. 하지만 그걸 듣고 "남자가 확실하지 않구나" 생각하면 안 돼요. 질문자가 누구인지 보고 참고해야지, 질문자 안 보고 참고하면 안 돼요. 엄마가 질문하면 아이 문제라고 하고, 남편이 질문하면 아내 문제라고 하는데, 그걸 듣고 "우리 엄마 때문이구나" 이렇게 생각하면 안 된다는 거예요.\n\n질문자: 네.\n\n스님: 인천 사람이 서울 가려면 동쪽으로 가라고 하는데, 서울 사람이 듣고 "나도 동쪽으로 가야겠다" 생각하면 안 돼요. 부모가 반대할 만하다는 걸 인정하고, 섭섭하게 생각하지 마세요. "어머니가 반대하는 건 당연하다고 생각합니다. 제가 선